### Part 1: Convert your previously written Beam pipeline(s) to Dataflow. Run them on Dataflow over the
entire input data; debug and fix as necessary.

Run over ENTIRE dataset

In [1]:
%run apartment_dataflow.py

/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1881: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = pcoll.pipeline.options.view_as(
/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery_file_loads.py:904: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = p.options.view_as(GoogleCloudOptions).temp_location


In [1]:
%run host_dataflow.py

/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1881: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = pcoll.pipeline.options.view_as(
/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery_file_loads.py:904: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = p.options.view_as(GoogleCloudOptions).temp_location


### Part 2: Verify that the BigQuery output tables from the previous step contain a valid primary & foreign key

Referential Integrity:
Primary key (listing_id, host_id) of apartment_dataflow table

Check the number of rows in the apartment_dataflow table

In [1]:
%%bigquery
SELECT count(*) FROM (SELECT listing_id, host_id FROM datamart.apartment_Dataflow)

,f0_
0,45127


Check the number of distince rows in the apartment_dataflow table
(the number should be the same as the previous number)

In [2]:
%%bigquery
SELECT DISTINCT count(*) FROM (SELECT listing_id, host_id FROM datamart.apartment_Dataflow)

,f0_
0,45127


----

Referential Integrity:Primary key (host_id) of host_dataflow table

Check the number of rows in the host_dataflow table

In [1]:
%%bigquery
SELECT count(*) host_id FROM datamart.host_Dataflow

,host_id
0,26318


Check the number of distinct rows in the host_dataflow table (the number should be the same as the previous number)

In [2]:
%%bigquery
SELECT DISTINCT count(*) host_id FROM datamart.host_Dataflow

,host_id
0,26318


---

Referential Integrity: Foreign Key (host_id) of apartmen_dataflow table
Checking FK (host_id) of apartment_Beam references PK (host_id) of host table (Result should be zero if the referential integrety of FK host_id is valid)

In [3]:
%%bigquery
SELECT count(host_id) FROM datamart.apartment_Dataflow WHERE host_id NOT IN (select host_id from datamart.host)

,f0_
0,0


### Part 3: Implement three cross-dataset queries in your milestone4 notebook:

SQL 1:
Compute the 5-year average home value (Zillow data) and compare that to the daily Airbnb renting price,categorized by room types for each different city
Plus, computing the home_vale-to-apartmentRent ratio for cross-city comparison

In [2]:
%%bigquery
CREATE VIEW reports.Homeval_Aptprice AS
SELECT home_value, city1 as city,AVG_Apt_price_day,Type_room, (home_value/AVG_Apt_price_day) as Homeval_to_apt_ratio FROM
(
# Home value
(SELECT AVG(home.home_value) as home_value, home.city as city1 FROM datamart.home_value as home WHERE extract (year from home.date) > 2015
GROUP BY city1)
# END: Home value
JOIN 
# Apt by room
(SELECT AVG(apt.price) as AVG_Apt_price_day, apt.city as city2, apt.room_type as Type_room FROM datamart.apartment_Dataflow as apt
GROUP BY Type_room, city2)
# END: Apt by room
ON city1=city2
)
ORDER BY city1 

,home_value,city,AVG_Apt_price_day,Type_room,Homeval_to_apt_ratio
0,338552.016129,Austin,68.894737,Shared room,4914.047599
1,338552.016129,Austin,112.047553,Private room,3021.502985
2,338552.016129,Austin,297.384326,Entire home/apt,1138.432617
3,338552.016129,Austin,647.200000,Hotel room,523.102621
4,471465.790323,Boston,178.192370,Entire home/apt,2645.824797
5,471465.790323,Boston,96.246704,Private room,4898.513602
6,471465.790323,Boston,185.520000,Hotel room,2541.320560
7,471465.790323,Boston,136.444444,Shared room,3455.368170
8,659523.580645,LA,108.691928,Private room,6067.824849
9,659523.580645,LA,282.457952,Entire home/apt,2334.944284


SQL 2: Compute the (percentage) increase in Long-term renting price over 3-year average for a different city (Zillow data) and compare with the % of Airbnb occupancy rate
for the calendar year 2020-2021 (only this year available)

In [1]:
%%bigquery
CREATE VIEW reports.Rent_Inc_Occ AS
SELECT AVG_Percent_Rent_Inc, city1 as city, occupy_rate FROM(
# Rent 3-Yar Avg Inc
SELECT AVG(percent_rent_increase) as AVG_Percent_Rent_Inc, zcity1 FROM(
# Rent Inc each Year
SELECT 100*(rent2 - rent1)/rent1 as percent_rent_increase, year1, year2, zcity1 
FROM
(SELECT AVG(lr1.rental) as rent1, extract (year from lr1.date) as year1, lr1.city as zcity1
FROM datamart.longrent as lr1 GROUP BY year1, lr1.city)
JOIN
(SELECT AVG(lr2.rental) as rent2, extract (year from lr2.date) as year2, lr2.city as zcity2
FROM datamart.longrent as lr2 GROUP BY year2, lr2.city)
ON year1+1=year2 and zcity1=zcity2)
WHERE year1>2017
# End: Rent Inc each Year
GROUP BY zcity1
# END: Rent 3-Yar Avg Inc
)
JOIN
(
# Occupancy
SELECT 100*(tot - not_use)/tot as occupy_rate, city1 FROM (
# Empty
(SELECT count(*) as not_use, apt.city as city1 FROM datamart.apartment_Dataflow as apt
WHERE apt.availability > 0 GROUP BY apt.city)
# End: Empty
JOIN 
# Total
(SELECT count(*) as tot, apt2.city as city2 FROM datamart.apartment_Dataflow as apt2 
GROUP BY apt2.city)
# End: Total
ON city1=city2)
# END: Occupancy
)
ON city1=zcity1



,AVG_Percent_Rent_Inc,city,occupy_rate
0,1.459405,LA,23.066554
1,1.539365,Austin,39.197741
2,-0.712818,Boston,19.174839


SQL3: Compute % changes in inventory in the last 3 years and compare to the avarage available days of Airbnb listings in the year 2020-2021 for 3 cities

In [5]:
%%bigquery
CREATE VIEW reports.Inventory_vs_Availability AS
SELECT airbnb_avg_availability, AVG(percent_inventory_increase) AS avg_change_inventory, city FROM
(
# Avg days avail
(SELECT AVG(availability) AS airbnb_avg_availability, a.city as city0 FROM datamart.apartment_Dataflow a GROUP BY city0)
# End: Avg days avail
JOIN
# Inv Inc
(SELECT 100*(inventory2 - inventory1)/inventory1 as percent_inventory_increase, inventory1, inventory2, year1, year2, city1 AS city
FROM
(SELECT AVG(li1.inventory) as inventory1, extract (year from li1.date) as year1, li1.city as city1
FROM datamart.inventory as li1
GROUP BY year1, li1.city )
JOIN
(SELECT AVG(li2.inventory) as inventory2, extract (year from li2.date) as year2, li2.city as city2
FROM datamart.inventory as li2
GROUP BY year2, li2.city)
ON year1+1=year2 and city1=city2)  
# End: Inv Inc
ON city0=city
)
GROUP BY city, airbnb_avg_availability

,airbnb_avg_availability,avg_change_inventory,city
0,206.184959,-8.083436,LA
1,386.896992,-3.007883,Austin
2,422.918498,-13.245386,Boston
